In [8]:
import pickle
import pandas as pd
import re

# function to read data from file
def read_tagging(file_name,language):
    path="../data/"+language+"/tagging/"+file_name+".conllu"
    data = pd.read_csv (path, sep = '\t',quoting=3, names=["POSITION","WORD","TAG"])
    return data

def extract_sentences_from_dataframe(df):
    sentences = ''
    for index, row in df.iterrows():
        word = row['WORD']
        if pd.notnull(word):  # Se la parola non è nulla
            if sentences:  # Se c'è già una frase, aggiungi uno spazio prima della nuova parola
                sentences += ' '
            sentences += word
    return sentences

#deserialize data from a file
def load_data(file_name,language):
    path="../data/"+language+"/"+file_name
    try: 
        file = open(path, 'rb') 
        data = pickle.load(file) 
        return data
    except: 
        print("Error in reading data")

In [9]:
#conversione di un dataframe che contine le frasi etichettate 
#in un dataframe che contiene solo le entità etichettate nella forma di quadrupe (tag, sentence number, start index, end index)
def extract_entities_from_dataframe(dataframe):
    entity_spans = []

    current_entity_span = None
    current_sentence_index = 0

    for index, row in dataframe.iterrows():
        word = row['WORD']
        tag = row['TAG']
        position = row['POSITION']

        if position == 0:  # Inizio di una nuova frase
            current_sentence_index += 1

        if tag != 'O':
            if tag.startswith('B-'):
                # Se inizia una nuova entità, chiudi quella corrente e inizia una nuova
                if current_entity_span is not None:
                    entity_spans.append(current_entity_span)
                current_entity_span = {'Tag': tag[2:], 'Sentence Number': current_sentence_index}
                current_entity_span['Start Index'] = index
                current_entity_span['End Index'] = index
            elif tag.startswith('I-'):
                # Aggiungi la parola all'entità corrente
                if current_entity_span is not None:
                    current_entity_span['End Index'] = index
            else:
                print("Errore: Tag non riconosciuto.")

        else:
            # Se il tag è "O" ma siamo all'interno di una serie di tag non "O", chiudi l'entità corrente
            if current_entity_span is not None:
                entity_spans.append(current_entity_span)
                current_entity_span = None

    # Aggiungo l'ultima entità se presente
    if current_entity_span is not None:
        entity_spans.append(current_entity_span)

    # Creazione del dataframe di output
    output_data = {'Tag': [], 'Sentence Number': [], 'Start Index': [], 'End Index': []}
    for entity_span in entity_spans:
        output_data['Tag'].append(entity_span['Tag'])
        output_data['Sentence Number'].append(entity_span['Sentence Number'])
        output_data['Start Index'].append(entity_span['Start Index'])
        output_data['End Index'].append(entity_span['End Index'])

    output_df = pd.DataFrame(output_data)

    return output_df

In [28]:
def calculate_accuracy(system_df, golden_df):
    # Unione i due dataframe per confrontare i tag
    merged_df = pd.merge(system_df, golden_df, left_index=True, right_index=True, suffixes=('_system', '_golden'))
    # Conteggio di tag corrispondenti uguali
    correct_tags = (merged_df['TAG_system'] == merged_df['TAG_golden']).sum()
    accuracy = correct_tags / len(system_df)
    accuracy_percent = round(accuracy * 100, 1)
    return accuracy_percent

In [11]:
def calculate_precision_recall(predicted_df, golden_df,entity_type="all"):
    # Unione delle entità predette e delle entità corrette
    merged_df = pd.merge(predicted_df, golden_df, how='outer', indicator=True)
    # Filtra per tipo di entità se specificato
    if entity_type!="all":
        merged_df=merged_df[(merged_df['Tag'] == entity_type)]
    # Calcolo dei true positives (TP), false positives (FP) e false negatives (FN)
    TP = merged_df[(merged_df['_merge'] == 'both')].shape[0]
    FP = merged_df[(merged_df['_merge'] == 'right_only')].shape[0]
    FN = merged_df[(merged_df['_merge'] == 'left_only')].shape[0]

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    precision_percent = round(precision * 100, 1)
    recall_percent = round(recall * 100, 1)

    return precision_percent, recall_percent

In [32]:
import pandas as pd
from datetime import datetime
#Scrive su file i risultati delle metriche di tutti gli algorithmi per tutte le lingue
def evaluate_and_save_results(languages, output_file, dataset_size):
    sep = "-" * 139 + "\n" 
    sep2= "#"*139 + "\n"
    header_format = "{:<4} | {:<7} | {:<5} | {:<5} | {:<5} | {:<10} | {:<8} | {:<7} | {:<8} | {:<7} | {:<8} | {:<7} | {:<8} | {:<7} |\n"
    row_format = "{:<4} | {:<7} | {:<5.1f} | {:<5.1f} | {:<5.1f} | {:<10} | {:<8.1f} | {:<7.1f} | {:<8.1f} | {:<7.1f} | {:<8.1f} | {:<7.1f} | {:<9.1f} | {:<8.1f} |\n"

   
    with open(output_file, "a") as file:
        file.write(sep2)
        file.write(f"Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        file.write(f"Dimensione del dataset: {dataset_size}\n")
        file.write(f"Tipo di Smoothing: "+ open('../data/it/tagging/smoothing_type', 'r').read()+"\n")
        file.write(sep)
        file.write("Risultati:\n")
        file.write(header_format.format("Lang", "Alg", "Acc", "Prec", "Rec", "", "LOC_Prec", "LOC_Rec", "PER_Prec", "PER_Rec", "ORG_Prec", "ORG_Rec", "MISC_Prec", "MISC_Rec"))
        
        file.write(sep)
        for language in languages:
            gold_df = read_tagging("golden_tag", language)[:dataset_size]
            gold_quadruples_df = extract_entities_from_dataframe(gold_df)
            for alg in ["naive","viterbi","memm"]:
                result_df = read_tagging(str(alg)+"_tag", language)[:dataset_size]
                result_quadruples = extract_entities_from_dataframe(result_df)
                metrics=[]
                metrics.append(calculate_accuracy(result_df, gold_df))
                for entity in ["all","LOC","PER","ORG","MISC"]:
                    precision, recall = calculate_precision_recall(result_quadruples, gold_quadruples_df,entity)
                    metrics.append(precision)
                    metrics.append(recall)
            
                file.write(row_format.format(language, alg, metrics[0], metrics[1], metrics[2], "", metrics[3], metrics[4], metrics[5], metrics[6], metrics[7], metrics[8], metrics[9], metrics[10]))
            file.write(sep)
        file.write(sep2+"\n")

# Esempio di utilizzo
languages = ["en"]#, "it", "es"]
output_file = "../evaluation_results.txt"
dataset_size = 2200 #numero di righe (token) da considerare

evaluate_and_save_results(languages, output_file, dataset_size)
